## Introduction

In this lab exercise, we are going to create a program that will break Diffie Hellman (DH) keys. The reason why we use Diffie Hellman and not RSA is that standard RSA requires a key-length of 1024 bits, which currently will require a lot of compute and time to break and not feasible for this lab.
The Diffie Hellman algorithm does not have this minimal requirement and "works" with smaller key-lengths. Smaller key-lengths "work" but that does not mean they are secure.

**Note**: Please add/edit your code in test.py and run it via a Terminal

### Alice & Bob
Alice and Bob would like to exchange a sensitive message message.
By generating a shared key using Diffie Hellman (DH) they are able to encrypt the sensitive message with their shared secret and prevent others from eaves dropping.

## General steps

Please follow these generic steps for each of the following sections.
* Copy the code into `test.py`
* This file has a reference to a library file `labfuncs.py` where the actual functions used are located
* Set a `bit_size` larger than 3
* Run the program: `python test.py` from a terminal in the lab1 directory


### Generate initial keys
There are specific requirements for DH thus we will write a program that will ensure this.

Alice will start the process:
* Alice generates `pub_prime_key`, a public prime number and `pub_base_key` a primitive root modulo of `pub_prime_key`.


***NOTE***: Do not copy `def main():` line this is just there to prevent processing of the Jupyter Notebook

In [3]:
def main():

    bit_size = 4

    print("Given the bit_size of", bit_size)
    print("    the Private keys of Alice & Bob will be")
    print("         larger than:", pow(2, bit_size - 1),"and")
    print("        smaller than:", pow(2, bit_size) - 1)

    pub_base_key = 0
    pub_prime_key = None
    primitive_roots = None
    while pub_base_key < 2:
        pub_prime_key = randPrime(bit_size)
        primitive_roots, pub_base_key = find_primitive_root(pub_prime_key)
    print("    the Public Prime key (p)")
    print("        is set to:", pub_prime_key)
    print("    the derived Public Base key (g) is")
    print("        selected from:", primitive_roots)
    print("        and set to:", pub_base_key)


# Exchanging keys and generating private keys

* Alice shares `pub_prime_key`, a prime number and `pub_base_key` in the clear with Bob
* Alice and Bob both generate a personal secret key (`priv_a, priv_b`), based on the same `bit_size`. We make sure that `priv_a`, the secret key of Alice is not the same as `priv_b`, the secret key of Bob.
* Alice and Bob both calculate their public key (`pub_a, pub_b`)

***NOTE***: Do not copy `def main():` line this is just there to prevent processing of the Jupyter Notebook


In [4]:
def main():

    # ...

    priv_a = rand_n(bitSize)
    priv_b = priv_a
    while priv_a == priv_b:
        priv_b = rand_n(bitSize)

    print("The Private key of")
    print("    Alice is:", priv_a)
    print("      Bob is:", priv_b)

    pub_a = calc_key(priv_a, pub_base_key, pub_prime_key)
    pub_b = calc_key(priv_b, pub_base_key, pub_prime_key)

    print("The Public key of")
    print("    Alice is:", pub_a)
    print("      Bob is:", pub_b)


## Calculating the shared Secret
In DH, the public and privates are *only* used to calculate the `shared_secret`.
This `shared_secret` is not communicated between Alice and Bob, but only used to encrypt messages using another algorithm like AES.
In RSA the public and private key can be used to exchange other messages, like a `shared_secret` and thus is communicated over the path between Alice and Bob.

In [5]:
def main():

    # ...

    shared_secret_a = calc_key(priv_a, pub_base_key, pub_prime_key)
    shared_secret_b = calc_key(priv_b, pub_base_key, pub_prime_key)

    if shared_secret_a != shared_secret_b:
        print("ERROR: Alice's & Bob's sharedSecret are not the same")
        raise SystemExit()

    print("The Shared secret of")
    print("    Alice is:", shared_secret_a)
    print("      Bob is:", shared_secret_b)


## Introducing Mallory
Mallory has been able to listen in on the conversation between Alice and Bob.
She has been able to collect the following public information exchanged:
* Alice messages containing
  * `pub_prime_key`
  * `pub_base_key`
  * Alice's public key `pub_a`
* Bob message containing
  * Bob's public key `pub_b`

Malory will now try to calculate the private keys of Alice and Bob and their shared secret.

In [6]:
def main():

    # ...
    print("Malory determines")
    mal_priv_key_a, mal_priv_key_b = find_secret_key(pub_a, pub_b, pub_prime_key, pub_base_key)

    print("    Alice's private key to be:", mal_priv_key_a)
    print("    Bob's private key to be:", mal_priv_key_b)

    mal_shared_secret_a = calc_key(mal_priv_key_a, pub_b, pub_prime_key)
    mal_shared_secret_b = calc_key(mal_priv_key_b, pub_a, pub_prime_key)

    print("    Alice's shared secret to be:", mal_shared_secret_a)
    print("    Bob's shared secret to be:", mal_shared_secret_b)


## Malory's challenge
The function `find_secret_key` that Malory uses to find the private keys of Alice and Bob is quite time consuming.

    a = pub_a
    b = pub_b
    p = pub_prime_key
    g = pub_base_key

***QUESTION*** Which variable or number could you change to speed up the search of this function?

    def find_secret_key(a, b, p, g):
        a_, b_ = None, None
        for x in range(1, p):
            if (g ** x) % p == a:
                a_ = x
            if (g ** x) % p == b:
                b_ = x
        return a_, b_

You are probably using small integers to perform these tasks and the process will become slower with `bit_size` increasing.
We need Quantum computation to speed up the process of this discrete logarithm problem.

Let's go to Lab2